Notebook Para Transcrever Vídeos Do YouTube via API com uso apenas de CPU.<p>
Criação:<p>
Luis Gustavo<p>
Email: lgsantini1@yahoo.com.br<p>
Adaptação:<p>
Daniel Dias<p>
Site: www.inarte.com.br<p>
Email: inarte@gmail.com <p>
Temos a Honra de Participar da Comunidade do Canal Sandeco https://www.youtube.com/@canalsandeco<p>

Instruções para Execução no Colab<p>
Execute cada bloco um a um aguarde cada bloco completar a execução completamente antes de passar para o próximo.<p>
Clique no botão "Iniciar Transcrição" para iniciar a transcrição e tradução do áudio no fim do último bloco.<p>
Procure o arquivo ao lado esquerdo na pasta onde ele foi salvo, e clique nos três pontos para realizar o download.<p>
Se deseja realizar mais de uma transcrição deve repetir todos os passos novamente.<p>
Pode começar suas transcrições!

In [ ]:
!pip install langchain_community youtube_transcript_api deep_translator pytube
print("Instalado com sucesso!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.8 MB/s eta 0:00:00
Instalado com sucesso!


Execute o próximo bloco de códigos e vá até o final para colocar o link do video que quer transcrever.

In [ ]:
import os
from langchain_community.document_loaders import YoutubeLoader
from youtube_transcript_api import NoTranscriptFound
from deep_translator import GoogleTranslator
import textwrap

def split_text(text, max_length):
    """Divide o texto em partes menores com no máximo max_length caracteres"""
    parts = []
    while len(text) > max_length:
        split_point = text.rfind(" ", 0, max_length)
        if split_point == -1:
            split_point = max_length
        parts.append(text[:split_point])
        text = text[split_point:].strip()
    parts.append(text)
    return parts

def format_text_for_a4(text, width=70):
    """Formata o texto para se ajustar a uma largura adequada para papel A4"""
    wrapper = textwrap.TextWrapper(width=width)
    word_list = wrapper.wrap(text=text)
    formatted_text = "\n".join(word_list)
    return formatted_text

def translate_text(text, source_lang, target_lang):
    """Traduz o texto de source_lang para target_lang"""
    parts = split_text(text, 4999)
    translated_text = ""
    for part in parts:
        translated_part = GoogleTranslator(source=source_lang, target=target_lang).translate(part)
        translated_text += translated_part + "\n"
    return translated_text

def load_youtube_video_info(url, original_language):
    # Inicializa o carregador do YouTube com a língua preferida como português
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True, language=original_language)

    try:
        # Carrega as informações do vídeo e a transcrição
        result = loader.load()

        if result:
            # Recupera informações de metadados
            author = result[0].metadata['author']
            title = result[0].metadata['title']
            length = result[0].metadata['length']
            # Verifica se a transcrição está disponível
            transcript = result[0].page_content if hasattr(result[0], 'page_content') else "Transcrição não disponível."

            if transcript != "Transcrição não disponível.":
                # Formata a transcrição original
                original_formatted = format_text_for_a4(transcript)

                # Traduz a transcrição para o outro idioma
                if original_language == 'pt':
                    translated_text = translate_text(transcript, 'pt', 'en')
                    translated_formatted = format_text_for_a4(translated_text)
                    info = (f"Vídeo encontrado de {author} com duração de {length} segundos.\n\n"
                            f"Título: {title}\n\n"
                            f"Transcrição (Português):\n{original_formatted}\n\n"
                            f"Transcrição (Inglês):\n{translated_formatted}")
                else:
                    translated_text = translate_text(transcript, 'en', 'pt')
                    translated_formatted = format_text_for_a4(translated_text)
                    info = (f"Vídeo encontrado de {author} com duração de {length} segundos.\n\n"
                            f"Título: {title}\n\n"
                            f"Transcrição (Inglês):\n{original_formatted}\n\n"
                            f"Transcrição (Português):\n{translated_formatted}")

                # Cria a pasta 'transcritos' se não existir
                if not os.path.exists("transcritos"):
                    os.makedirs("transcritos")

                # Salva a saída em um arquivo .txt na pasta 'transcritos'
                filename = f"transcritos/{author}_{title}.txt".replace(" ", "_").replace(".", "_").replace("-", "_")
                with open(filename, "w", encoding="utf-8") as file:
                    file.write(info)

                # Retorna a informação e o nome do arquivo
                return info, filename
            else:
                return "Transcrição não disponível.", None
        else:
            return "Nenhum resultado encontrado.", None
    except NoTranscriptFound:
        return "Transcrição não encontrada para este vídeo.", None

# Solicitar a URL do YouTube e o idioma original
url = input("Digite a URL do YouTube e aperte Enter: ")
original_language = input("Digite o idioma original do vídeo (pt para português, en para inglês): ")

# Processar a transcrição e salvar o arquivo
info, filename = load_youtube_video_info(url, original_language)
print(info)

if filename:
    print(f"Transcrição salva em: {filename}")
    from google.colab import files
    files.download(filename)


Digite a URL do YouTube e aperte Enter: https://youtu.be/Yp0RRTdtkqY
Digite o idioma original do vídeo (pt para português, en para inglês): pt
Vídeo encontrado de Felipe Rocha • dicasparadevs com duração de 2754 segundos.

Título: Curso de Git & GitHub Para Completos Iniciantes

Transcrição (Português):
e fala galera seja muito bem-vinda mais um vídeo aqui do canal dicas
para deves o canal que traz para você programação Sem enrolação e hoje
sem enrolação Vamos aprender sobre kit e derrame essas duas
ferramentas que você precisa saber tanto para usar nos seus projetos
pessoais quanto para usar em vagas de emprego Beleza então hoje eu vou
ensinar para vocês em relação na qualidade que você já está acostumado
aqui no nosso canal mas antes eu peço para você que se você gostar do
vídeo Se você ficar até o final gostar achar que agregam algum valor
em você eu peço para que você por favor deixe o seu joinha embaixo e
se inscreva no canal porque isso ajuda muito a gente aqui a continuar
trazen

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>